In [282]:
# imports
from jinja2 import Environment, FileSystemLoader
from pprint import pprint
import re

In [283]:
# funções gerais
class FuncoesUteis():
    @staticmethod
    def to_camel_case(s:str)->str:
        """converte snakecase to camelcase"""
        parts = s.lower().split('_')
        return parts[0] + ''.join(word.capitalize() for word in parts[1:])
 

    @staticmethod
    def calcular_tamanho_componente_label(texto: str) -> int:
        """
        Calcula o tamanho de um componente de label com base no texto fornecido.
        """
        total = 0
        texto = FuncoesUteis.convert_ascii_codes(texto)
        for char in texto:
            if char.isupper():  # Verifica se é maiúscula
                total +=  10
            elif char in 'mMwW': # verifica se são letras mais extensas
                total +=  11
            elif char in 'iljrIyf':  # Verifica se são letras mais estreitas
                total +=  4
            elif char == ' ': # verifica se é espaço
                total += 11
            else:  # Qualquer outro caractere
                total +=  8
        
        return total + 10
    
    @staticmethod
    def add_tamanho_tfCheck(tamanho:int):
        return tamanho + 20
    
    @staticmethod
    def convert_ascii_codes(input_string):
        
        def replace_code(match:re.Match)->str:
            """Função de substituição que converte '#n' para o caractere ASCII correspondente"""
            # Extrai o número do padrão '#n'
            ascii_code = int(match.group(1))
            # Converte o código ASCII para o caractere correspondente
            return chr(ascii_code)
        
        # Usa regex para encontrar todos os padrões '#n' e substituí-los pelo caractere correspondente
        adjusted_string = re.sub(r"'#(\d+)'", replace_code, input_string)
        
        return adjusted_string

In [284]:
from re import Pattern

class TbData():
    dados:list = []
    texto:str = ""
    def __init__(self, texto:str):
        #converter string para dicionario
        self.dados = self.converterStringToDicionario(texto)
        self.processarNovosCampos()
        self.texto = texto

    def converterStringToDicionario(self, texto: str) -> list:
        # Define um padrão regex para identificar objetos com seus nomes e tipos
        object_pattern: Pattern = re.compile(r"object (\w+): (\w+).*?>.*?end", re.DOTALL)
        
        # Define um padrão regex para identificar a seção FieldDefs e seu conteúdo
        fielddefs_pattern: Pattern = re.compile(r"FieldDefs\s*=\s*<\s*(.*?)\s*>\n\s*", re.DOTALL)
        
        # Define um padrão regex para identificar itens dentro da seção FieldDefs
        item_pattern: Pattern = re.compile(r"item\s+(.*?)\send", re.DOTALL)
        
        # Define um padrão regex para identificar pares chave-valor em cada item
        attr_pattern: Pattern = re.compile(r"(\w+) = '*(.*?)'*$", re.MULTILINE)
        
        objects = []  # Lista que armazenará todos os objetos convertidos em dicionários
        
        # Itera sobre todos os objetos encontrados no texto usando o padrão de objeto
        for obj_match in object_pattern.finditer(texto):
            # Extrai o nome e tipo do objeto da correspondência
            obj_name = obj_match.group(1)
            obj_type = obj_match.group(2)
            
            # Extrai o texto completo do objeto correspondente
            obj_texto = obj_match.group(0)
            
            # Cria um dicionário para o objeto atual, inicializando a chave 'FieldDefs' com uma lista vazia
            current_object = {'name': obj_name, 'type': obj_type, 'FieldDefs': []}
            
            # Itera sobre todos os itens encontrados no texto do objeto usando o padrão de item
            for item_match in item_pattern.finditer(obj_texto):
                current_item = {}  # Dicionário que armazenará os pares chave-valor para o item atual
                item_text = item_match.group(1)  # Extrai o texto do item
                
                # Extrai pares chave-valor do texto do item usando o padrão de atributo
                for attr_match in attr_pattern.finditer(item_text):
                    key, value = attr_match.group(1), attr_match.group(2)
                    current_item[key] = value  # Adiciona o par chave-valor ao dicionário do item

                # Adiciona o item processado à lista 'FieldDefs' do objeto atual
                current_object['FieldDefs'].append(current_item)
            

            # ordena a lista de objetos 'FieldDefs' de acordo com a linha e coluna
            current_object['FieldDefs'] = sorted(current_object['FieldDefs'], key=lambda x: (int(x.get('lin', 9999)), int(x.get('col', 9999))))

            # Remove a seção FieldDefs do texto do objeto para processar atributos restantes
            remaining_attrs = re.sub(fielddefs_pattern, '', obj_texto)
            
            # Itera sobre os pares chave-valor restantes no texto do objeto
            for attr_object_match in attr_pattern.finditer(remaining_attrs):
                key, value = attr_object_match.group(1), attr_object_match.group(2)
                current_object[key] = value  # Adiciona os pares chave-valor restantes ao dicionário do objeto
            
            # Adiciona o dicionário do objeto à lista de objetos
            objects.append(current_object)
        
        # Retorna a lista de objetos convertidos em dicionários
        return objects
    
    def processarNovosCampos(self):
        # calcula o tamanho da label do campo
        for index, dado in enumerate(self.dados):
            # calcula cada label
            maior_label = max([FuncoesUteis.calcular_tamanho_componente_label(a.get('Caption')) for a in dado.get('FieldDefs') if a.get('col') == '1'])
            # adiciona o tamanho do label ao componente
            # dado["FieldDefs"] = [{**item, "tamanhoLabel": maior_label} if (item.get('col') == '1' and item.get('FieldType') not in ['ftCheck']) else {**item, "tamanhoLabel": FuncoesUteis.calcular_tamanho_componente_label(item.get('Caption'))} for item  in dado["FieldDefs"]]
            for item in dado["FieldDefs"]:
                if item["FieldType"] == "ftCheck":
                    item["tamanhoComponente"] = FuncoesUteis.calcular_tamanho_componente_label(item.get('Caption')) + 10
                    maior_label = max([FuncoesUteis.calcular_tamanho_componente_label(a.get('Caption')) for a in dado.get('FieldDefs') if a.get('col') == '1' and a.get('lin') > item["lin"]], [])
                else:
                    item["tamanhoComponente"] = maior_label
    

                                                                                          




In [285]:
from io import TextIOWrapper
#abre o arquivo
arquivoTabela:TextIOWrapper = open('data/tbPergunta.txt', 'r')
textoTabela = arquivoTabela.read()
arquivoTabela.close()

# inicializando variáveis da tabela
tbData:TbData = TbData(textoTabela)
#pprint(tbData.dados)

In [286]:
def processarNovosCampos(tbData:TbData):
        # calcula o tamanho da label do campo
        for index, dado in enumerate(tbData.dados):
            # calcula cada label
            maior_label = max([FuncoesUteis.calcular_tamanho_componente_label(a.get('Caption')) for a in dado.get('FieldDefs') if a.get('col') == '1'])
            # adiciona o tamanho do label ao componente com listcompreension, porém ficou complexo demais ai criei um for
            # dado["FieldDefs"] = [{**item, "tamanhoLabel": maior_label} if (item.get('col') == '1' and item.get('FieldType') not in ['ftCheck']) else {**item, "tamanhoLabel": FuncoesUteis.calcular_tamanho_componente_label(item.get('Caption'))} for item  in dado["FieldDefs"]]
            for item in dado["FieldDefs"]:
                if item["FieldType"] == "ftCheck":
                    item["tamanhoLabel"] = FuncoesUteis.calcular_tamanho_componente_label(item.get('Caption')) + 10
                    maior_label = max([FuncoesUteis.calcular_tamanho_componente_label(a.get('Caption')) for a in dado.get('FieldDefs') if a.get('col') == '1' and a.get('lin') > item["lin"]], [])
                else:
                    item["tamanhoLabel"] = maior_label

processarNovosCampos(tbData)
pprint(tbData.dados)

[{'Cursor': 'CRM_PERGUNTAS',
  'DeltaMode': 'dmChanged',
  'FieldDefs': [{'Calculated': 'False',
                 'Caption': 'campo tipo tfString',
                 'FieldType': 'ftString',
                 'Name': 'TF_STRING',
                 'NullOnEmpty': 'False',
                 'PrimaryKey': 'True',
                 'Updatable': 'False',
                 'WOrigem': 'EhNone',
                 'WOwner': 'FrWizard',
                 'col': '1',
                 'lin': '1',
                 'tamanhoComponente': 160,
                 'tamanhoLabel': 160},
                {'Calculated': 'False',
                 'Caption': 'campo tipo tfInteger',
                 'FieldType': 'ftInteger',
                 'Name': 'TF_INTEGER',
                 'NullOnEmpty': 'False',
                 'PrimaryKey': 'True',
                 'Updatable': 'False',
                 'WOrigem': 'EhNone',
                 'WOwner': 'FrWizard',
                 'col': '2',
                 'lin': '1',
        

In [287]:
# Configura o carregador de templates para a pasta do script
env = Environment(loader=FileSystemLoader('templates'))
env.trim_blocks = True
env.lstrip_blocks = True


#add as filter to jinja
env.filters['to_camel_case'] = FuncoesUteis.to_camel_case

#add as filter to jinja
env.filters['calcular_tamanho_componente_label'] = FuncoesUteis.calcular_tamanho_componente_label

env.filters['add_tamanho_tfCheck'] = FuncoesUteis.add_tamanho_tfCheck

#renderizando o template
template = env.get_template('template_init.jinja')
print(template.render(data = tbData.dados))

object groupboxtbPerguntas: TFGroupbox
  Width = 745
  Height = 450
  Caption = 'Detalhes tbPerguntas'
  Font.Charset = DEFAULT_CHARSET
  Font.Color = clWindowText
  Font.Height = -11
  Font.Name = 'Tahoma'
  Font.Style = []
  Padding.Top = 0
  Padding.Left = 0
  Padding.Right = 0
  Padding.Bottom = 0
  ParentFont = False
  TabOrder = 0
  Margin.Top = 0
  Margin.Left = 0
  Margin.Right = 0
  Margin.Bottom = 0
  Flex.Vflex = ftTrue
  Flex.Hflex = ftTrue
  WOwner = FrInterno
  WOrigem = EhNone
  Scrollable = False
  Closable = False
  Closed = False
  Orient = coHorizontal
  Style = grp3D
  HeaderImageId = 0
  object vboxDetalhetbPerguntas: TFVBox
    Left = 2
    Top = 15
    Width = 745
    Height = 450
    Align = alClient
    AutoWrap = False
    BevelKind = bkTile
    BevelOuter = bvNone
    BorderStyle = stNone
    Caption = ''
    FlowStyle = fsTopBottomLeftRight
    Padding.Top = 5
    Padding.Left = 5
    Padding.Right = 10
    Padding.Bottom = 5
    TabOrder = 0
    Margin.Top 

In [288]:
def convert_ascii_codes(input_string):
    # Função de substituição que converte '#n' para o caractere ASCII correspondente
    def replace_code(match):
        # Extrai o número do padrão '#n'
        ascii_code = int(match.group(1))
        # Converte o código ASCII para o caractere correspondente
        return chr(ascii_code)
    
    # Usa regex para encontrar todos os padrões '#n' e substituí-los pelo caractere correspondente
    adjusted_string = re.sub(r"'#(\d+)'", replace_code, input_string)
    
    return adjusted_string

def calcular_tamanho_componente_label(texto: str) -> int:
    total = 0
    
    for char in texto:
        if char.isupper():  # Verifica se é maiúscula
            total +=  10
        elif char in 'iljrIyf':  # Verifica se é i, l ou j
            total +=  4
        elif char == ' ':
            total += 9
        else:  # Qualquer outro caractere
            total +=  8
    
    return total + 10


# Exemplo de uso
texto = "Cód. Questionario"
resultado = calcular_tamanho_componente_label(texto)
print(convert_ascii_codes("'C'#243'd. Questionario'"))
print(resultado)

'Cód. Questionario'
139


In [289]:
meudicionario = [{"nome": "teste1", "campos": {"a": 1, "b": 2}},
                 {"nome": "teste2", "campos": {"a": 1, "b": 2}}]

# Adicionando o campo "c" com valor 3 a todos os dicionários dentro de "campos"
meudicionario = [{**item, "campos": {**item["campos"], "c": 3}} for item in meudicionario]

print(meudicionario)

[{'nome': 'teste1', 'campos': {'a': 1, 'b': 2, 'c': 3}}, {'nome': 'teste2', 'campos': {'a': 1, 'b': 2, 'c': 3}}]
